In [ ]:
import datetime
import dash
from dash import html,dash_table, dcc
import numpy as np
import pandas as pd

from dash.dependencies import Output, Input, State
from flask_caching.backends import FileSystemCache
from dash_extensions.callback import CallbackCache,DiskCache
import plotly.express as px

from sqlalchemy import create_engine
import mysql.connector #  pip3 install mysql-connector-python
import configparser as cp

from jupyter_dash import JupyterDash
host_name='ec2-13-58-144-138.us-east-2.compute.amazonaws.com'

In [ ]:
app=JupyterDash(__name__)

## 01 db connection ##
config=cp.ConfigParser()
config.read('/home/ubuntu/cert/db_login.txt')
db_config=config['ivan_db']


# 2. db connection
engine=create_engine('mysql+mysqlconnector://{0:s}:{1:s}@{2:s}/{3:s}'.format(db_config['userid'],
                                                                             db_config['pwd'],
                                                                             db_config['hostname'],
                                                                             'STOCK_PRED'
                                                                            ))
    
    


app.layout=html.Div([
    html.H1('Stock Price Analysis - Daily Update'),
    #html.H3('Stock data as of {}; data cached at {} UTC'.format(stock_details_df().Date.max().date(),
    #                                                            stock_details_df().cache_time.max().strftime('%Y-%m-%d %H:%M:%S')
    #                                                           )),
    html.Div([
        html.Div([
            html.Button('Refresh with Cache',id='btn_cache'),
            html.Div(id='stock_data_refresh_text'),
            html.Div(id='model_data_refresh_text'),
        ]),
        
        html.Div([
            dcc.Dropdown(
                #options=reg_result().Stock.unique(),
                value='CRM',
                id='pick-stock')
        ],style={'display':'inline-block','width':'49%','float':'right'}),
        
        html.Div([
            dcc.Input(id='r_squared_input',name='R Squared', value=0.8, type='number', placeholder='R_squared'), # R_squared
            dcc.Input(id='coef_input', value=0, type='number', placeholder='Coef'),
            dcc.Input(id='growth_rate_input',value=0, type='number', placeholder='Growth_rate'),
            html.Div(id='filters_input')
        ]), # ,style={'display':'inline-block','width':'49%','float':'left'}
        



        
        
    ]),
    #html.Div(id='stock_data_refresh_text'),
    #html.Div(id='model_data_refresh_text'),
    
    html.Div([
        html.Div([
            html.H3('Linear Regression Output - Last 60 Days'),
            dash_table.DataTable(id='linear_reg_summary',
                                 columns=[{'name':i,'id':i} for i in ['Stock','Name','Industry','R_squared','Coef','start_price','end_price','growth_rate']],
                                 page_size=10
                                )
        ],style={'padding-top':'5px'}),
        html.Div([
            dcc.Graph(id='coef_r_scatter',
                      #figure=px.scatter(df_lr.loc[(df_lr.Coef>-5)&(df_lr.growth_rate<2),:],
                      #                  x='R_squared',y='Coef',
                      #                  color='growth_rate',size='end_price',
                      #                  hover_data=['Stock'],
                      #                  title='R_squared and Coef Distribution'
                      #                 )
                     )
        ],style={'padding-top':'5px'})
    ],style={'float':'left','width':'49%'}),
        

    
    ## right ##
    html.Div([
        dcc.Loading(dcc.Graph(id='stock_price_line_l40')), # revising dcc.Loading
        dcc.Graph(id='stock_price_line') 
        
    ],style={'float':'right','width':'49%'}),
    
    
    ## loading cache
    dcc.Loading(dcc.Store(id='stock_details_df'),
                fullscreen=True,
                type='dot'
               ),
    dcc.Loading(dcc.Store(id='reg_result'),
                fullscreen=True,
                type='dot'
               )
    
])





## cache - config - start  ##
#cache=Cache(app.server,config={
#    'CACHE_TYPE':'filesystem',
#    'CACHE_DIR':'/home/ubuntu/projects/Stock_Price_Prediction/dashboard/cache_dir/'
#})
#TIMEOUT = 60*30 # 30 minutes

# cache -1: overall stock price data: there are two df & df_l40 in use, need to consolidate to one
#@cache.memoize(timeout=TIMEOUT)

## Data Refresh ##

cc=CallbackCache(cache=DiskCache(cache_dir=str("/home/ubuntu/projects/Stock_Price_Prediction/dashboard/disk_cache")
                                )) # DiskCache '/home/ubuntu/projects/Stock_Price_Prediction/dashboard/disk_cache/'


@cc.cached_callback(Output('stock_details_df','data'),
                    [Input('btn_cache','n_clicks')]
                   )
def pull_stock_details(n_clicks):
    df=pd.read_sql("""SELECT *
                      FROM STOCK_PRED.ALL_STOCK_HIST
                      WHERE DATE>=CURDATE()-INTERVAL 360 DAY""",
                   con=engine)
    df.loc[:,'Close_rf']=[round(x,0) for x in df.Close]
    df.loc[:,'cache_time']=datetime.datetime.now()
    return df



# cache -2: linear regression results
@cc.cached_callback(Output('reg_result','data'),
                    [Input('btn_cache','n_clicks')]
                   )
def pull_reg_result(n_clicks):
    df_lr=pd.read_sql("""SELECT *
                         FROM STOCK_PRED.LINEAR_REG_L40
                         WHERE MODEL_DATE IN (SELECT MAX(MODEL_DATE) FROM STOCK_PRED.LINEAR_REG_L40)
                         ORDER BY MODEL_DATE DESC, WT_COEF DESC""",
                         con=engine
                     )

    ## data cleaning
    df_lr1=df_lr.loc[:, ['Model_date','Stock','Name','Industry','R_squared','Coef','P_values',
                         'start_price','end_price','Num_records_dist','growth_rate','WT_Coef']]
    df_lr1.loc[:,'Name']=[x[:50] for x in df_lr1.Name]
    df_lr1.loc[df_lr1.Industry.isnull(),'Industry']='Not Available'
    df_lr1.loc[:,'Industry']= [x[:30] for x in df_lr1.Industry]


    df_lr1.loc[:,'R_squared']=round(df_lr1.R_squared,2)
    df_lr1.loc[:,'Coef']=round(df_lr1.Coef,2)
    df_lr1.loc[:,'start_price']=round(df_lr1.start_price,2)
    df_lr1.loc[:,'end_price']=round(df_lr1.end_price,2)
    df_lr1.loc[:,'growth_rate']=round(df_lr1.growth_rate,2)
    df_lr1.loc[:,'P_values']=round(df_lr1.P_values,2)

    #df_lr1.loc[:,'Model_date']=df_lr1.Model_date.dt.date

    df_lr1.drop(['Num_records_dist'],axis=1,inplace=True)
    
    return df_lr1




## interaction ##

@cc.callback(Output('pick-stock','options'),
             [Input('reg_result','data')]
            )
def insert_options(df):
    return df.Stock.unique()



@cc.callback([Output('linear_reg_summary','data')],
             [Input('reg_result','data'), 
              Input('r_squared_input','value'),
              Input('coef_input','value'),
              Input('growth_rate_input','value')]
)
def update_linear_reg_tabel(df_lr1,
                            r_squared_thres,coef_thres,growth_rate_thres):
    df_lr1_filtered=df_lr1.loc[(df_lr1.R_squared>=r_squared_thres)&
                               (df_lr1.Coef>=coef_thres)&
                               (df_lr1.growth_rate>=growth_rate_thres) ,:].reset_index(drop=True)
    df_lr1_filtered.loc[:,'WT_Coef']=round(df_lr1_filtered.WT_Coef,2)
    df_lr1_filtered.drop('Model_date',axis=1,inplace=True)
    
    return df_lr1_filtered.loc[:,['Stock','Name','Industry','R_squared','Coef','start_price','end_price','growth_rate']].to_dict('records')


@cc.callback(
    Output('filters_input','children'),
    [Input('r_squared_input','value'),
     Input('coef_input','value'),
     Input('growth_rate_input','value')]
)
def update_filters_input(r_squared_thres,coef_thres,growth_rate_thres):
    return 'Filter the data by: R Squared >= {:.2f} & Coefficient >= {:.2f} & Growth Rate >= {:.1f}%'.format(r_squared_thres,
                                                                                                             coef_thres,
                                                                                                             growth_rate_thres*100
                                                                                                            )



@cc.callback(
    Output('stock_price_line_l40','figure'),
    [Input('stock_details_df','data'), # extract data from cache
     Input('pick-stock','value')]
)
def update_l40_trend(df,pick_stock):
    df_l40=df.loc[df.Date>=df.Date.max()-datetime.timedelta(days=60),:]
    fig_l40=px.line(df_l40.loc[df_l40.Stock==pick_stock,:],
                    x='Date',y='Close',
                    text='Close_rf',
                    title='{} Last 60 Days Stock Price'.format(pick_stock)
                   
                   )
    return fig_l40


@cc.callback(
    Output('stock_price_line','figure'),
    [Input('stock_details_df','data'), # extract data from cache
     Input('pick-stock','value')]
)
def update_la_trend(df,pick_stock):
    fig_la=px.line(df.loc[df.Stock==pick_stock,:],
                   x='Date',y='Close',
                   title='{} Last 1 Year Stock Price'.format(pick_stock)
                  )
    return fig_la



@cc.callback(
    Output('coef_r_scatter','figure'),
    [Input('reg_result','data'),
     Input('pick-stock','value'),
     Input('r_squared_input','value'),
     Input('coef_input','value'),
     Input('growth_rate_input','value')]
    
)
def highlight_scatter_plot(df_lr1,
                           pick_stock,r_squared_thres,coef_thres,growth_rate_thres
                          ):
    
    selected_stocks=df_lr1.loc[(df_lr1.R_squared>=r_squared_thres)&
                               (df_lr1.Coef>=coef_thres)&
                               (df_lr1.growth_rate>=growth_rate_thres),'Stock']
    
    df_scatter_clean=df_lr1.loc[(df_lr1.Coef>-5)&
                                (df_lr1.Coef<=10)&
                                (df_lr1.growth_rate<2),:].reset_index(drop=True)
    
    fig_scatter=px.scatter(df_scatter_clean,
                           x='R_squared',y='Coef',
                           #color='growth_rate',
                           #size='end_price',
                           #opacity=0.5,
                           hover_data=['Stock'],
                           title='R_squared and Coef Distribution')
    
    #fig_scatter['data'][0]['marker']['color']=['red' if c==pick_stock else 'grey' for c in df_lr.loc[(df_lr.Coef>-5)&(df_lr.growth_rate<2),'Stock']]
    fig_scatter['data'][0]['marker']['color']=np.where([c==pick_stock for c in df_scatter_clean.Stock],'red',
                                                       np.where([c in selected_stocks.values for c in df_scatter_clean.Stock],
                                                                'blue',
                                                                'grey')
                                                      )
    fig_scatter['data'][0]['marker']['size']=[20 if c==pick_stock else 10 for c in df_scatter_clean.Stock]
    ## reference: https://community.plotly.com/t/how-to-highlight-a-single-bar-on-select-in-plotly-dash/60739
    ## figure data structure: https://plotly.com/python/figure-structure/
    
    return fig_scatter


## text field ##
@cc.callback(
    Output('stock_data_refresh_text','children'),
    Input('stock_details_df','data')
)
def stock_df_refresh_time(df):
    return 'Stock data is updated as of {:}'.format(df.Date.max().strftime('%Y-%m-%d'))

@cc.callback(
    Output('model_data_refresh_text','children'),
    Input('reg_result','data')
)
def stock_df_refresh_time(df_lr1):
    return 'Model data is updated as of {:}'.format(df_lr1.Model_date.max().strftime('%Y-%m-%d %H:%M:%S'))



# Registers the callbacks on the application
cc.register(app)

    


if __name__=='__main__':
    app.run_server(mode='external',
                   host=host_name,
                   debug=True
                  )